<h1> Multi Agnet Systems</h1>

<h2>Problem defenition</h2>

---

> <p>Vacuum cleaner problem is a well-known search problem for an agent which 
works on Artificial Intelligence. In this problem, our vacuum cleaner is our agent. It is a goal based agent, and the goal of this agent, which is the vacuum cleaner, is to clean up the whole area. So, in the classical vacuum cleaner problem, we have two rooms and one vacuum cleaner. There is dirt in both the rooms and it is to be cleaned. The vacuum cleaner is present in any one of these rooms. So, we have to reach a state in which both the rooms are clean and are dust free.</p>

---

In [1]:
import numpy as np 
import random
import threading
from time import sleep

In [2]:
# Color code
red = '\033[91m'
green = '\033[92m'
yellow = '\033[93m' 
blue = '\033[94m'
pink = '\033[95m'
teal = '\033[96m'
grey = '\033[97m'

In [3]:
global env_matrix
global agent_list
global agent_pos 
global agent_head

In [4]:
class Environment:

    def __init__(self, n = 4):

        global env_matrix
        global agent_list
        global agent_pos
        global agent_head

        # initialize an n by n environmnet (default = 4)
        # each cell has two states
        # 0 : clean cell
        # 1 : dirty cell
        # at first all of the cells are clean (np.zeros)
        env_matrix = np.zeros((n, n), dtype=np.int8)

        # agnet list = [A1, A2, ...]
        agent_list = []
        # agent(s) positions = [[x1, y1], [x2, y2], ...]
        agent_pos = []
        # agent(s) Head = ['U', 'L'] -> U: up, D: down, L: left, R: right
        agent_head = []

        # env size
        self.__size = n
       
    def random_environment(self):
        # select some cells to be in the dirty state
        global env_matrix
        rnd_mat = np.random.randint(0, 2, size = (self.__size, self.__size))
        env_matrix = env_matrix + rnd_mat
        
        
    def add_agent(self, num, percept_error):

        global agent_list
        global agent_pos
        global agent_head

        # set a random position for each agnet
        x = np.random.randint(0, self.__size)
        y = np.random.randint(0, self.__size)
        agent_pos.append([x, y])

        # set head randomly
        Head = random.choice('UDRL')
        agent_head.append(Head)

        # add an agent to environment
        A = Agent(self.__size, num, percept_error) # still need to complete
        agent_list.append(A)
        del A

    def winner(self, suction0, suction1, cost0, cost1):
        if (suction0 > suction1):
            return 0
        elif(suction0 < suction1):
            return 1
        else:
            if(cost0 > cost1):
                return 0
            else:
                return 1

    def start_simulation(self,debug = False):
        global agent_list
        # agent(s) start to cleaning the environment
        # simulation ends when agents figure out the environmet is clear
        A = agent_list[0]
        B = agent_list[1]
        t1 = threading.Thread(target=A.start, args=(debug,))
        t2 = threading.Thread(target=B.start, args=(debug,))

        # starting thread 1
        t1.start()
        # starting thread 2
        t2.start()
 
        t1.join()
        t2.join()
        print(pink, 'After the process:')
        show()

        cost0 = agent_list[0].get_cost()
        cost1 = agent_list[1].get_cost()
        suction0 = agent_list[0].num_of_suction()
        suction1 = agent_list[1].num_of_suction()

        
        print(teal, 'agent 0 Suction: ', suction0)
        print(teal, 'agent 1 Suction: ', suction1)

        print(teal, 'agent 0 Cost: ', cost0)
        print(teal, 'agent 1 Cost: ', cost1)

        print(yellow, 'winner is: ', 
              self.winner(suction0, suction1, cost0, cost1))

        print(pink, 'the agent 0 path:')
        print(agent_list[0].print_path())
        print(blue, 'the agent 1 path:')
        print(agent_list[1].print_path())

        
    

In [5]:
class Agent:
    def __init__(self, env_size, num, percept_error): 
        self.__idx = num
        self.__env_size = env_size
        self.__cost = 0
        self.__selected_path = np.random.randint(0,3)
        self.__num_of_suck = 0
        self.__the_agent_path = []
        self.__percept_error = percept_error

    def get_pos(self):
        global agent_pos
        global agent_head

        return (agent_pos[self.__idx], agent_head[self.__idx])


    def move(self):

        self.__cost += 1

        (x,y), head = self.get_pos()
        
        if (head == 'U'):
            x = x - 1
            if (x >= 0):
                agent_pos[self.__idx] = [x, y]

        elif (head == 'D'):
            x = x + 1
            if (x < self.__env_size):
                agent_pos[self.__idx] = [x, y]

        elif (head == 'R'):
            y = y + 1
            if (y < self.__env_size):
                agent_pos[self.__idx] = [x, y]

        elif (head == 'L'):
            y = y - 1
            if (y >= 0):
                agent_pos[self.__idx] = [x,y]

    def percept(self):
        global env_matrix
        
        (x,y), head = self.get_pos()

        if (env_matrix[x, y] == 1):
            # Dirty

            r = random.random()
            if (r < self.__percept_error):
                # error
                return False
            else:
                return True
        else:
            # Clean
            r = random.random()
            if (r < self.__percept_error):
                # Error
                return True
            else:
                return False

    def suck(self):
        global env_matrix

        self.__cost += 2
        (x,y), head = self.get_pos()

        env_matrix[x ,y] = 0 
        self.__num_of_suck += 1

    def start(self,debug = False):
        global agent_pos

        self.__cost = 0

        path = [
                [[0,0],[0,1],[0,2],[0,3],
                 [1,3],[1,2],[1,1],[2,1],
                 [2,2],[2,3],[3,3],[3,2],
                 [3,1],[3,0],[2,0],[1,0]],
                
                [[0,0],[1,0],[2,0],[3,0],
                 [3,1],[2,1],[1,1],[1,2],
                 [2,2],[3,2],[3,3],[2,3],
                 [1,3],[0,3],[0,2],[0,1]],
                
                [[0,0],[0,1],[0,2],[0,3],
                 [1,3],[1,2],[2,2],[2,3],
                 [3,3],[3,2],[3,1],[3,0],
                 [2,0],[2,1],[1,1],[1,0]]
                ]

        (x,y), head = self.get_pos()

        home_idx = path[self.__selected_path].index([x,y])
        current_idx = home_idx
        next_idx = home_idx + 1

        if(next_idx == 16):
            next_idx = 0
        
        while (next_idx != home_idx):
            if(debug == True):
                show()
                sleep(0.1)
            else:
                sleep(0.1)

            if(self.percept()):
                self.suck()
            

            # check that the other agent is not in the next idx
            if (agent_pos[1 - self.__idx] == path[self.__selected_path][next_idx]):
                continue 

            self.go_A2B(path[self.__selected_path][current_idx],
                        path[self.__selected_path][next_idx], debug)

            # save the agent path
            self.__the_agent_path.append(
                path[self.__selected_path][current_idx])

            current_idx = next_idx 
            next_idx = next_idx + 1


            if(next_idx == 16):
                next_idx = 0

            # check if finish
            if (self.is_clear()):
                break



    def go_A2B(self, pos1, pos2, debug = False):

        global agent_head

        x1, y1 = pos1
        x2, y2 = pos2

        if(x1 == x2):
            # Horizontal move
            if (y2 > y1):
                # Right
                while(agent_head[self.__idx] != 'R'):
                    self.turn(debug)
                self.move()
                
            elif (y1 > y2):
                # Left
                while(agent_head[self.__idx] != 'L'):
                    self.turn(debug)
                self.move()

        elif (y1 == y2):
            # vertical move
            if (x2 > x1):
                # Down
                while(agent_head[self.__idx] != 'D'):
                    self.turn(debug)
                self.move()
                
            elif (x1 > x2):
                # Up
                while(agent_head[self.__idx] != 'U'):
                    self.turn(debug)
                self.move()

    def turn(self, debug = False):
        global agent_head
        
        if (agent_head[self.__idx] == 'U'):
            agent_head[self.__idx] = 'R'
        elif(agent_head[self.__idx] == 'R'):
            agent_head[self.__idx] = 'D'
        elif(agent_head[self.__idx] == 'D'):
            agent_head[self.__idx] = 'L'
        elif(agent_head[self.__idx] == 'L'):
            agent_head[self.__idx] = 'U'

        if (debug == True):
            show()
            sleep(0.1)
        else:
            sleep(0.1)

    def get_cost(self):
        return self.__cost

    def is_clear(self):
        global env_matrix

        if (env_matrix.sum() == 0):
            return True
        else:
            return False

    def num_of_suction(self):
        return self.__num_of_suck

    def print_path(self):
        print(self.__the_agent_path)

In [6]:
def show():

        global env_matrix
        global agent_pos
        global agent_head

        # show the environment in a suitable way
        # states
        # agent(s)
        # using color print

        print(green, 'Dirty: *')
        print(blue, 'Agents: #')
        print(red, 'Agent\'s Head: (U: up, D: down, R: right, L: left)')
        print()
        
        H_line = ((8 * 4) + 1) * '-'
        print(yellow, H_line)

        for i in range(4):
            print(yellow, '|', end='')

            for j in range(4):
                if(env_matrix[i,j] == 1):
                    # cell is Dirty
                    # print *
                    print(green, '*', end='')
                else:
                    print(green,' ', end='')

                if ([i, j] in agent_pos):
                    # position of agents
                    # print #
                    #print(blue, '#', end='')
                    print(blue, agent_pos.index([i,j]), end='')
                    print(red, agent_head[agent_pos.index([i,j])], end='')
                else:
                    print(blue,' ', end='')
                    print(red, ' ', end='')

                # end of the cell
                print(yellow, '|', end='')

            print() # new line after each line
            print(yellow, H_line)

In [7]:
env = Environment(4)
env.random_environment()
env.add_agent(0, 0.3)
env.add_agent(1, 0.3)
show()

 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |       | *     |       |       |
 ---------------------------------
 | * 0 L |   1 U |       |       |
 ---------------------------------
 | *     |       | *     |       |
 ---------------------------------
 | *     |       | *     | *     |
 ---------------------------------


In [8]:
env.start_simulation(debug = True)

 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |       | *     |       |       |
 ---------------------------------
 | * 0 L |   1 U |       |       |
 ---------------------------------
 | *     |       | *     |       |
 ---------------------------------
 | *     |       | *     | *     |
 ---------------------------------
 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |       | *     |       |       |
 ---------------------------------
 | * 0 L |   1 U |       |       |
 ---------------------------------
 | *     |       | *     |       |
 ---------------------------------
 | *     |       | *     | *     |
 ---------------------------------
 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |       | *     |       |       |
 ---------------------------------
 | * 0 L |   1 U |       |       |
 ----------

In [7]:
# analysis:
for i in range(3): 
    env = Environment(4)
    env.random_environment()
    env.add_agent(0, 0.0)
    env.add_agent(1, 0.0)
    show()
    env.start_simulation(False)
    print(teal, 50 * '/')
    del env

 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 | *     | *     |       | * 1 D |
 ---------------------------------
 | *     |       | *     |       |
 ---------------------------------
 |       | *     | *     |       |
 ---------------------------------
 |   0 R |       |       |       |
 ---------------------------------
 After the process:
 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |       |       |       |       |
 ---------------------------------
 |       |       |   0 L |       |
 ---------------------------------
 |       |       |       |       |
 ---------------------------------
 |       |       |   1 D |       |
 ---------------------------------
 agent 0 Suction:  1
 agent 1 Suction:  6
 agent 0 Cost:  12
 agent 1 Cost:  24
 winner is:  1
 the agent 0 path:
[[3, 0], [2, 0], [2, 1], [1, 1], [1, 0], [0, 0], [0, 1], [0, 2], [0, 3], [1, 3]]
None
 the

In [23]:
# analysis:
for i in range(3): 
    env = Environment(4)
    env.random_environment()
    env.add_agent(0, 0.1)
    env.add_agent(1, 0.1)
    show()
    env.start_simulation(False)
    print(teal, 50 * '/')
    del env

 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |       |       |       | *     |
 ---------------------------------
 |       |   0 R | *     | *     |
 ---------------------------------
 | *     |       |       |       |
 ---------------------------------
 |       |       |   1 U |       |
 ---------------------------------
 After the process:
 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |       |   0 L |       |       |
 ---------------------------------
 |       |       |       |       |
 ---------------------------------
 |       |       |       |       |
 ---------------------------------
 |   1 D |       |       |       |
 ---------------------------------
 agent 0 Suction:  2
 agent 1 Suction:  3
 agent 0 Cost:  13
 agent 1 Cost:  16
 winner is:  1
 the agent 0 path:
[[1, 1], [1, 2], [2, 2], [3, 2], [3, 3], [2, 3], [1, 3], [0, 3], [0, 2]]
None
 the agent 1

In [7]:
# analysis:
for i in range(3): 
    env = Environment(4)
    env.random_environment()
    env.add_agent(0, 0.3)
    env.add_agent(1, 0.3)
    show()
    env.start_simulation(False)
    print(teal, 50 * '/')
    del env

 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 | *     | *     | *     |       |
 ---------------------------------
 | *     |       | *     | *     |
 ---------------------------------
 | *     | *     | *     |       |
 ---------------------------------
 | * 0 R |       |       | * 1 L |
 ---------------------------------
 After the process:
 Dirty: *
 Agents: #
 Agent's Head: (U: up, D: down, R: right, L: left)

 ---------------------------------
 |   0 L |       |       |       |
 ---------------------------------
 |       |       |   1 R |       |
 ---------------------------------
 |       |       |       |       |
 ---------------------------------
 |       |       |       |       |
 ---------------------------------
 agent 0 Suction:  8
 agent 1 Suction:  8
 agent 0 Cost:  29
 agent 1 Cost:  29
 winner is:  1
 the agent 0 path:
[[3, 0], [3, 1], [2, 1], [1, 1], [1, 2], [2, 2], [3, 2], [3, 3], [2, 3], [1, 3], [0, 3], [